<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_6_pubmed_bigtweet_mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'
100  9115  100  9115    0     0  28219      0 --:--:-- --:--:-- --:--:-- 28132
curl: (23) Failed writing body (290 != 1213)


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [50]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,13,It's 6 am and this adderall got me reevaluatin...
1,84,paraplegic
2,91,jitters
3,189,dizziness
4,189,dizzy


In [51]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-6.test.csv",header=None)
test.head()

,0,1
0,13,awake in the middle of the night for an hour or 2
1,24,heart beat for being too strong
2,74,60% of my success is due to taking Adderall
3,85,tremor
4,94,anxiety


In [52]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [54]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [55]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [60]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.221870,4.645652,0.248512,15:46


In [61]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.476030,4.303285,0.282282,15:48


In [62]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.162329,4.262953,0.286561,15:50


In [63]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.107154,4.233830,0.289351,15:49


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [65]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.115623,6.909988,0.053571,00:22


In [66]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.241100,5.527600,0.062500,00:27


In [67]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.767251,5.167567,0.142857,01:04


In [68]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.526365,4.908134,0.160714,01:06
1,5.349925,4.828950,0.178571,01:08


In [69]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.118599,4.548225,0.196429,01:07
1,4.997751,4.430864,0.214286,01:05


In [70]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.791038,4.352716,0.232143,01:06
1,4.645858,4.345920,0.241071,01:05


In [71]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.505001,4.241729,0.223214,01:06
1,4.366954,4.217880,0.267857,01:09


In [72]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.213947,4.168698,0.276786,01:06
1,4.096331,4.160772,0.241071,01:04


In [73]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.933721,4.069029,0.276786,01:05
1,3.898406,4.038914,0.285714,01:03
2,3.728139,4.049320,0.294643,01:08
3,3.557168,4.055180,0.276786,01:06


In [74]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.427527,4.069569,0.285714,01:07
1,3.412208,4.210823,0.250000,01:07
2,3.267446,4.080525,0.285714,01:07
3,3.111249,4.091587,0.276786,01:10


In [75]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.961937,4.033451,0.303571,01:07
1,2.969631,4.145009,0.276786,01:08
2,2.827863,4.065363,0.285714,01:06
3,2.739654,4.115613,0.285714,01:03


In [76]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.553283,4.228691,0.258929,01:08
1,2.611714,4.241327,0.294643,01:06
2,2.535302,4.082767,0.294643,01:05
3,2.432420,4.095346,0.294643,01:07


In [77]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.295006,4.079320,0.294643,01:06
1,2.340710,4.260068,0.294643,01:05
2,2.267422,4.265625,0.294643,01:06
3,2.167578,4.286864,0.294643,01:10


In [78]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.108301,4.291182,0.267857,01:06
1,2.148473,4.451974,0.276786,01:04
2,2.060306,4.387471,0.285714,01:06
3,2.005553,4.417266,0.294643,01:08


In [79]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.953024,4.457770,0.303571,01:07
1,1.989026,4.508485,0.294643,01:08
2,1.903114,4.612628,0.294643,01:10
3,1.816756,4.580234,0.303571,01:03


In [80]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.844727,4.588376,0.285714,01:06
1,1.903966,4.756202,0.285714,01:04
2,1.894337,4.805915,0.303571,01:04
3,1.739146,4.687196,0.303571,01:07


In [81]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.757369,4.710628,0.303571,01:05
1,1.769848,4.765004,0.276786,01:10
2,1.746754,4.801931,0.285714,01:06
3,1.700163,4.756584,0.285714,01:07


In [82]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.698883,4.616758,0.276786,01:06
1,1.690222,4.717345,0.267857,01:07


In [83]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.779529,4.978623,0.294643,01:09
1,1.698424,4.980334,0.294643,01:04


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [85]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [86]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[2150, 1767, 1599, 13, 978, 1096, 189, 1265, 189, 1531, 224, 1767, 815, 233, 233, 2147, 749, 1925, 1599, 1136, 231, 532, 426, 1253, 1253, 706, 1265, 749, 1265, 1136, 792, 809, 819, 1529, 1265, 824, 1265, 2017, 908, 231, 923, 1265, 974, 1669, 978, 978, 561, 978, 1936, 231, 1669, 187, 1767, 1099, 1769, 1509, 1265, 396, 1787, 1265, 712, 1373, 1265, 1767, 1265, 1265, 231, 1529, 1295, 1265, 1308, 1325, 2149, 1340, 720, 1356, 1356, 712, 1925, 1711, 1718, 1718, 1597, 1597, 233, 1597, 1610, 1610, 1265, 1265, 1787, 1099, 1787, 2032, 265, 1727, 1727, 1265, 1265, 231, 749, 749, 1718, 1787, 2032, 1265, 1787, 1265, 231, 978, 56, 824, 1130, 2103, 1936, 1936, 233, 1936, 815, 1265, 233, 231, 2147, 452, 2018, 1265, 2032, 2147, 2069, 1508, 1508, 2149, 1265, 2147, 1265, 1196, 233, 2147, 2147, 2147, 2149, 2194, 2194]
[13, 24, 74, 85, 94, 187, 189, 189, 189, 217, 224, 224, 224, 233, 233, 233, 265, 355, 392, 396, 426, 532, 561, 600, 600, 706, 744, 749, 755, 791, 792, 809, 818, 818, 819, 825, 852, 85

In [87]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

41
0.2867132867132867
